In [103]:
import os #
import numpy as np #
import pandas as pd #
import multiprocessing as mp #
from itertools import permutations #

#-----CLASSIFIER------
from sklearn.svm import SVC #
from sklearn import metrics #
from sklearn.ensemble import RandomForestClassifier #
from sklearn.model_selection import RandomizedSearchCV #

In [102]:
#-----read FMBA metadata------
FMBA_metadata=pd.read_csv("../fmba/fmba_metadata_edited.tsv", sep=",",header=0, dtype = str)
print("FMBA_metadata \n", FMBA_metadata.COVID_status.value_counts())

#-----read Adaptive metadata------
AB_metadata=pd.read_csv("../adaptive/adaptive-metadata-edited.tsv",sep="\t")
AB_metadata["sample_short_name"]=AB_metadata["sample_name"].apply(lambda x: x[:-5]) #remove _TCRB from end of names
print("\nAdaptive_metadata \n", AB_metadata["COVID-19-status"].value_counts())

FMBA_metadata 
 COVID       1061
healthy      433
precovid     118
unknown       27
Name: COVID_status, dtype: int64

Adaptive_metadata 
 acute        1140
recovered     239
baseline       74
exposed        26
non-acute       4
Name: COVID-19-status, dtype: int64


In [73]:
#----METADATA-----
#----select FMBA cohorts-----
fmba_COVID=set(FMBA_metadata.loc[FMBA_metadata['COVID_status']=='COVID', "name"])
print("fmba_COVID:", len(fmba_COVID))
fmba_healthy=set(FMBA_metadata.loc[FMBA_metadata['COVID_status']=='healthy', "name"])
print("fmba_healthy:", len(fmba_healthy))
fmba_precovid=set(FMBA_metadata.loc[FMBA_metadata['COVID_status']=='precovid', "name"])
print("fmba_precovid:", len(fmba_precovid))

#----select Adaptive cohorts-----
adaptive_acute=set(AB_metadata.loc[AB_metadata["COVID-19-status"]=="acute","sample_short_name"])
print("\nAdaptive_acute:",  len(adaptive_acute))
HIP=set(pd.read_csv("/projects/fmba_covid/COV_V_usage_adjustment_count_equation_v3/HIP_functional_adjusted/metadata.txt", sep="\t", header=None)[0])
HIP={xT.split(".")[0] for xT in HIP}
print("HIP:",  len(HIP))
KECK=set(pd.read_csv("/projects/fmba_covid/COV_V_usage_adjustment_count_equation_v3/KECK_functional_adjusted/metadata.txt", sep="\t", header=None)[0])
KECK={xT.split("_")[0] for xT in KECK}
print("KECK:",  len(KECK))

fmba_COVID: 1061
fmba_healthy: 433
fmba_precovid: 118

Adaptive_acute: 1140
HIP: 665
KECK: 120


In [89]:
#-----sequencing runs-----
FMBA_metadata["sequencingDate"].value_counts()

2020 10 19    566
2020 12 18    377
2020 09 05    192
2020 09 28    191
2020 09 27    189
2021 01 07    118
2021 02 13      6
Name: sequencingDate, dtype: int64

In [49]:
#-----datasets-----
AB_metadata=AB_metadata.loc[AB_metadata["COVID-19-status"]=="acute"]
AB_metadata["Dataset"].value_counts()

COVID-19-DLS                     433
COVID-19-NIH/NIAID               357
COVID-19-HUniv12Oct              177
COVID-19-ISB                      69
COVID-19-IRST/AUSL                64
COVID-19-Adaptive                 37
COVID-19-Adaptive-MIRAMatched      3
Name: Dataset, dtype: int64

In [124]:
fmba_set1_={"2020 09 27", "2020 12 18", "2020 10 19"}
fmba_set2_={"2020 09 05", "2020 09 28", "2021 01 07"} # 2021 01 07==precovid
fmba_set1_=set(FMBA_metadata.loc[FMBA_metadata["sequencingDate"].isin(fmba_set1_), "name"])
fmba_set2_=set(FMBA_metadata.loc[FMBA_metadata["sequencingDate"].isin(fmba_set2_), "name"])

adaptive_set1_={"COVID-19-NIH/NIAID", "COVID-19-HUniv12Oct", "COVID-19-Adaptive", "KECK"}
adaptive_set2_={"COVID-19-DLS", "COVID-19-ISB", "COVID-19-IRST/AUSL", "HIP"}
adaptive_set1_=set(AB_metadata.loc[AB_metadata["Dataset"].isin(adaptive_set1_), "sample_short_name"]) | KECK
adaptive_set2_=set(AB_metadata.loc[AB_metadata["Dataset"].isin(adaptive_set2_), "sample_short_name"]) | HIP

In [ ]:
#-----read Adaptive and HIP data-----
AB_data=pd.read_csv("../CDR3_code/BLOSUM62/UNWEIGHT_TABLES/Adaptive_top_2000AA/Adaptive_top_10000AA.tsv",
                    sep="\t", index_col=0)
AB_data.index=[xT.split("_")[0] for xT in AB_data.index]

HIP_data=pd.read_csv("../CDR3_code/BLOSUM62/UNWEIGHT_TABLES/HIP_top_2000AA/HIP_top_10000AA.tsv",
                    sep="\t", index_col=0)
HIP_data.index=[xT.split(".")[0] for xT in HIP_data.index]

KECK_data=pd.read_csv("../CDR3_code/BLOSUM62/UNWEIGHT_TABLES/KECK_top_2000AA/KECK_top_10000AA.tsv",
                    sep="\t", index_col=0)
KECK_data.index=[xT.split("_")[0] for xT in KECK_data.index]

#----read FMBA data------
FMBA_data=pd.read_csv("../CDR3_code/BLOSUM62/UNWEIGHT_TABLES/FMBA_top_2000AA/FMBA_top_10000AA.tsv",
                    sep="\t", index_col=0)
FMBA_data.index=list(map(lambda x: x[0 : (x.rfind("S")-1)], FMBA_data.index))

In [296]:
#-----STATUS-----
print("number of samples in data:\n")

#-----FMBA data-----
FMBA_data_=FMBA_data.loc[FMBA_data.index.isin(fmba_COVID) | FMBA_data.index.isin(fmba_healthy) | FMBA_data.index.isin(fmba_precovid)]
FMBA_data_.loc[FMBA_data_.index.isin(fmba_COVID) , "status"]="COVID"
FMBA_data_.loc[FMBA_data_.index.isin(fmba_healthy) , "status"]="healthy"
FMBA_data_.loc[FMBA_data_.index.isin(fmba_precovid) , "status"]="healthy"
fmba_set1=FMBA_data_.loc[FMBA_data_.index.isin(fmba_set1_)]
print("fmba_set1", len(fmba_set1))
print("fmba_set1_COVID", len(fmba_set1.loc[fmba_set1.index.isin(fmba_COVID) | fmba_set1.index.isin(fmba_precovid)]))
print("fmba_set1_HEALTHY", len(fmba_set1.loc[fmba_set1.index.isin(fmba_healthy)]))
fmba_set2=FMBA_data_.loc[FMBA_data_.index.isin(fmba_set2_)]
print("fmba_set2", len(fmba_set2))
print("fmba_set2_COVID", len(fmba_set2.loc[fmba_set2.index.isin(fmba_COVID)]))
print("fmba_set2_HEALTHY", len(fmba_set2.loc[fmba_set2.index.isin(fmba_precovid)]))

#-----Adaptive data-----
AB_data_=AB_data.loc[AB_data.index.isin(adaptive_acute)]
AB_data_["status"]="COVID"
HIP_data["status"]="healthy"
KECK_data["status"]="healthy"
AB_data_=pd.concat([AB_data_, HIP_data, KECK_data])
adaptive_set1=AB_data_.loc[AB_data_.index.isin(adaptive_set1_)]
print("adaptive_set1", len(adaptive_set1))
print("adaptive_set1_COVID", len(adaptive_set1.loc[adaptive_set1.index.isin(adaptive_set1_-KECK)]))
print("adaptive_set1_HEALTHY", len(adaptive_set1.loc[adaptive_set1.index.isin(KECK)]))
adaptive_set2=AB_data_.loc[AB_data_.index.isin(adaptive_set2_)]
print("adaptive_set2", len(adaptive_set2))
print("adaptive_set2_COVID", len(adaptive_set2.loc[adaptive_set2.index.isin(adaptive_set2_-HIP)]))
print("adaptive_set2_HEALTHY", len(adaptive_set2.loc[adaptive_set2.index.isin(HIP)]))

number of samples in data:

fmba_set1 394
fmba_set1_COVID 147
fmba_set1_HEALTHY 247
fmba_set2 207
fmba_set2_COVID 107
fmba_set2_HEALTHY 100


/home/daria/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/daria/.local/lib/python3.6/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/home/daria/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

adaptive_set1 517
adaptive_set1_COVID 397
adaptive_set1_HEALTHY 120
adaptive_set2 1203
adaptive_set2_COVID 541
adaptive_set2_HEALTHY 662


In [297]:
#-----combinations of training and test sets-----
model_permutation_=list(permutations(("fmba_set1","fmba_set2", "adaptive_set1","adaptive_set2"), 2))
model_permutation=[]
for i in range(len(model_permutation_)):
    x, y =iter(model_permutation_[i])
    model_permutation.append(f'{x}/{y}')

print("training and test sets:\n")
model_permutation

training and test sets:



['fmba_set1/fmba_set2',
 'fmba_set1/adaptive_set1',
 'fmba_set1/adaptive_set2',
 'fmba_set2/fmba_set1',
 'fmba_set2/adaptive_set1',
 'fmba_set2/adaptive_set2',
 'adaptive_set1/fmba_set1',
 'adaptive_set1/fmba_set2',
 'adaptive_set1/adaptive_set2',
 'adaptive_set2/fmba_set1',
 'adaptive_set2/fmba_set2',
 'adaptive_set2/adaptive_set1']

In [ ]:
#-----SVM------
output_folder="model_comparison/"
output_file=output_folder+"/"+"model_comparison.tsv"

parameters = {"C": [0.01, 0.5, 1], "kernel": ["linear", "rbf"], "degree": [1, 2, 3], "gamma": ["scale"]} 
column_names=[*["normalization", "features", "mismatch", "weight"], *model_permutation]
datasets=list(permutations((fmba_set1, fmba_set2, adaptive_set1, adaptive_set2), 2))

def run_classifier(normalization, features, mismatch, weight,
                   datasets=datasets, parameters=parameters, column_names=column_names,
                   output_folder=output_folder, output_file=output_file):   
    scores=[]
    best_parameters=[]
    for dataset in datasets:
        train_set, test_set=iter(dataset)
#------split into features matrix X and vector with ansvers y-----
        X_to_fit=train_set[train_set.columns[~train_set.columns.isin(["status"])]]
        y_to_fit=train_set["status"]
        X_test=test_set[test_set.columns[~test_set.columns.isin(["status"])]]
        y_test=test_set["status"]
#-----find hyperparameters----
        clf=RandomizedSearchCV(SVC(class_weight="balanced"), parameters, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
        clf.fit(X_to_fit, y_to_fit)
        #-----best estimator-----
        SVM=clf.best_estimator_
        best_parameter=clf.best_params_
        best_parameters.append(best_parameter)
        y_pred=SVM.predict(X_test)
        score=SVM.score(X_test, y_test)
        scores.append(score)
    scores_and_features=[*[normalization, features, mismatch, weight], *scores]   
#-----make output file, if it doesn't exist-----
    try:
        os.mkdir(output_folder)
        with open(output_file,"w") as out_file:
            out_file.write("\t".join(column_names))
    except:
        pass
    #-----write a string to a file-----
    with open(output_file,"a") as out_file:
            out_file.write("\n")
            out_file.write("\t".join(str(xT) for xT in scores_and_features)) 
    return best_parameters         

In [298]:
#-----RANDOM FOREST-----
output_folder="model_comparison/"
output_file=output_folder+"/"+"model_comparison.tsv"

parameters = {'criterion': ['gini'], 'max_depth' : [3, 5, 7], 'min_samples_leaf': [4, 10, 20],
               'min_samples_split': [10, 20, 40], 'max_features': [15, 20, 50]}
column_names=[*["normalization", "features", "mismatch", "weight"], *model_permutation]
datasets=list(permutations((fmba_set1, fmba_set2, adaptive_set1, adaptive_set2), 2))

def run_classifier(normalization, features, mismatch, weight,
                   datasets=datasets, parameters=parameters, column_names=column_names,
                   output_folder=output_folder, output_file=output_file):   
    scores=[]
    best_parameters=[]
    for dataset in datasets:
        train_set, test_set=iter(dataset)
#------split into features matrix X and vector with ansvers y-----
        X_to_fit=train_set[train_set.columns[~train_set.columns.isin(["status"])]]
        y_to_fit=train_set["status"]
        X_test=test_set[test_set.columns[~test_set.columns.isin(["status"])]]
        y_test=test_set["status"]
#-----find hyperparameters----
        clf=RandomizedSearchCV(RandomForestClassifier(class_weight="balanced"), parameters, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
        clf.fit(X_to_fit, y_to_fit)
        #-----best estimator-----
        RF=clf.best_estimator_
        best_parameter=clf.best_params_
        best_parameters.append(best_parameter)
        y_pred=RF.predict(X_test)
        score=RF.score(X_test, y_test)
        scores.append(score)
    scores_and_features=[*[normalization, features, mismatch, weight], *scores]   
#-----make output file, if it doesn't exist-----
    try:
        os.mkdir(output_folder)
        with open(output_file,"w") as out_file:
            out_file.write("\t".join(column_names))
    except:
        pass
    #-----write a string to a file-----
    with open(output_file,"a") as out_file:
            out_file.write("\n")
            out_file.write("\t".join(str(xT) for xT in scores_and_features)) 
    return best_parameters         

In [299]:
run_classifier(normalization="top-2000 public", features="public clonotypes", mismatch="blosum", weight="unweighted")

Fitting 3 folds for each of 81 candidates, totalling 243 fits


/home/daria/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 81 is smaller than n_iter=100. Running 81 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:    8.4s finished
/home/daria/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 81 is smaller than n_iter=100. Running 81 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.


Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:    4.8s finished
/home/daria/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 81 is smaller than n_iter=100. Running 81 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.


Fitting 3 folds for each of 81 candidates, totalling 243 fits


...

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:    8.6s finished
/home/daria/.local/lib/python3.6/site-packages/sklearn/model_selection/_search.py:282: UserWarning: The total space of parameters 81 is smaller than n_iter=100. Running 81 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.


Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed:    8.8s finished


[{'min_samples_split': 20,
  'min_samples_leaf': 20,
  'max_features': 20,
  'max_depth': 5,
  'criterion': 'gini'},
 {'min_samples_split': 40,
  'min_samples_leaf': 4,
  'max_features': 20,
  'max_depth': 3,
  'criterion': 'gini'},
 {'min_samples_split': 20,
  'min_samples_leaf': 20,
  'max_features': 20,
  'max_depth': 7,
  'criterion': 'gini'},
 {'min_samples_split': 20,
  'min_samples_leaf': 4,
  'max_features': 15,
  'max_depth': 7,
  'criterion': 'gini'},
 {'min_samples_split': 20,
  'min_samples_leaf': 4,
  'max_features': 15,
  'max_depth': 3,
  'criterion': 'gini'},
 {'min_samples_split': 40,
  'min_samples_leaf': 4,
  'max_features': 15,
  'max_depth': 3,
  'criterion': 'gini'},
 {'min_samples_split': 20,
  'min_samples_leaf': 4,
  'max_features': 50,
  'max_depth': 3,
  'criterion': 'gini'},
 {'min_samples_split': 40,
  'min_samples_leaf': 4,
  'max_features': 50,
  'max_depth': 7,
  'criterion': 'gini'},
 {'min_samples_split': 20,
  'min_samples_leaf': 20,
  'max_features':

In [300]:
pd.read_csv("model_comparison/model_comparison.tsv", sep='\t')

,normalization,features,mismatch,weight,fmba_set1/fmba_set2,fmba_set1/adaptive_set1,fmba_set1/adaptive_set2,fmba_set2/fmba_set1,fmba_set2/adaptive_set1,fmba_set2/adaptive_set2,adaptive_set1/fmba_set1,adaptive_set1/fmba_set2,adaptive_set1/adaptive_set2,adaptive_set2/fmba_set1,adaptive_set2/fmba_set2,adaptive_set2/adaptive_set1
0,top-5000,enriched in COVID,single mismatch,weighted,0.859903,0.802708,0.507066,0.685279,0.996132,0.942643,0.626904,0.483092,1.000000,0.626904,0.483092,1.000000
1,top-5000,enriched in COVID,single mismatch,unweighted,0.827206,0.647687,0.576923,0.633205,0.770463,0.484452,0.432432,0.621324,0.495090,0.469112,0.595588,0.718861
2,top-5000,public clonotypes,single mismatch,weighted,0.878676,0.854093,0.653028,0.604247,0.761566,0.405074,0.432432,0.621324,0.811784,0.484556,0.426471,0.884342
3,top-5000,public clonotypes,single mismatch,unweighted,0.871324,0.807829,0.707038,0.648649,0.750890,0.711948,0.432432,0.621324,0.853519,0.430502,0.602941,0.898577
4,top-2000 public,enriched in COVID,single mismatch,weighted,0.850242,0.649903,0.446384,0.703046,0.740812,0.440565,0.373096,0.516908,0.733167,0.403553,0.463768,0.878143
5,top-2000 public,enriched in COVID,single mismatch,unweighted,0.855072,0.727273,0.518703,0.675127,0.657640,0.419784,0.373096,0.516908,0.553616,0.436548,0.487923,0.808511
6,top-2000 public,public clonotypes,single mismatch,weighted,0.855072,0.742747,0.458022,0.667513,0.744681,0.469659,0.373096,0.516908,0.812136,0.444162,0.328502,0.856867
7,top-2000 public,public clonotypes,single mismatch,unweighted,0.855072,0.775629,0.470490,0.687817,0.758221,0.746467,0.373096,0.516908,0.814630,0.373096,0.483092,0.870406
8,full repertoire,public clonotypes,without mismatch,weighted,0.869565,0.221477,0.540650,0.572337,0.243289,0.545528,0.454690,0.680124,0.481301,0.453100,0.686335,0.887584
9,full repertoire,public clonotypes,without mismatch,unweighted,0.838509,0.224832,0.541463,0.562798,0.250000,0.548780,0.454690,0.680124,0.581301,0.454690,0.680124,0.753356
